In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
import utils as utils
from fractions import Fraction
from func import Func
import func

from IPython.display import display, HTML

# Inline plotting
%matplotlib inline

# Make sympy print pretty math expressions
sy.init_printing()

utils.load_custom_styles()

# Exercise 1

<img src="figures/exam-jan-2018/ex1.png" width="800" />















---
<img src="figures/exam-jan-2018/ex1a.png" width="800" />



<img src="figures/exam-jan-2018/sol-ex1a.jpg" width="500" />


---
<img src="figures/exam-jan-2018/ex1b.png" width="800" />







Using following procedure:

- Step 1: Convert inequalities to equalites using slack variables
- Step 2: Convert objective function to an equally.
- Step 3: Construct the initial simplex tableaux
- Step 4: Find the pivot column. The pivot column can be found by looking at the smallest value at the bottom row. In this case, we have two pivot columns and we select the first. Next, find the pivot row. Pick the smallest of smallest of $b_i / a_{i1}$. 
- Step 5: Make the pivot column to a basis column

<img src="figures/exam-jan-2018/sol-ex1b.jpg" width="500" />


































In [1]:
from simplex import SimplexSolver 
A = [[1, 2],
     [1, 5],
     [1, 0]]
b = [12, 25, 6]
c = [1, 1]
solver = SimplexSolver()
res = solver.run_simplex(A, b, c, prob='max', enable_msg=True, latex_path=None)

   x_1   x_2   s_1   s_2   s_3    b    
|   1     2     1     0     0    12   | s_1
|   1     5     0     1     0    25   | s_2
|   1     0     0     0     1     6   | s_3
|  -1    -1     0     0     0     0   |
Current solution: {'x_1': 0, 'x_2': 0, 's_1': Fraction(12, 1), 's_2': Fraction(25, 1), 's_3': Fraction(6, 1), 'z': 0}

   x_1   x_2   s_1   s_2   s_3    b    
|   1     2     1     0     0    12   | s_1
|   1     5     0     1     0    25   | s_2
|   1     0     0     0     1     6   | s_3
|  -1    -1     0     0     0     0   |

There are negative elements in the bottom row, so the current solution is not optimal. Thus, pivot to improve the current solution. The entering variable is x_1 and the departing variable is s_3.


Perform elementary row operations until the pivot is one and all other elements in the entering column are zero.

   x_1   x_2   s_1   s_2   s_3    b    
|   0     2     1     0    -1     6   | s_1
|   0     5     0     1    -1    19   | s_2
|   1     0     

In [2]:
res

{'x_1': Fraction(6, 1),
 'x_2': Fraction(3, 1),
 's_1': 0,
 's_2': Fraction(4, 1),
 's_3': 0,
 'z': Fraction(9, 1)}

The maximum for $f$ can be found using SciPy Linear Programming algorithm:

In [2]:
import linprog as lp

A = [[1, 2],
     [1, 5],
     [1, 0]]
b = [12, 25, 6]
c = [1, 1]

res = lp.maximise(A, b, c)
lp.pretty_print(res)

<IPython.core.display.Math object>

The maximum is $f(6, 3) = 6+3=9$.

---
<img src="figures/exam-jan-2018/ex1c.png" width="800" />








We know that at least one of the extreme points or corners of the feasible area corresponds to an optimal solution:

<img src="figures/exam-jan-2018/theorem-6.png" width="800" />





Changing inequality ii) moves only one of the extreme points; from $(0, 5)$ to $(0, 6)$. This is because only the $y$-intersect is changed and not the slope of the corresponding line.

Evaluating our objective function on this new extreme point, we get $f(0, 6) = 0+6 = 6$. This does not change the maximum for $f$ which was $f(6, 3)=6+3=9$

---
# Exercise 2

<img src="figures/exam-jan-2018/ex2a.png" width="800" />















The gradient of $f$:
$$
\nabla f = 
\begin{bmatrix}
\frac{\partial f}{\partial x_1} \\
\frac{\partial f}{\partial x_2} \\
\frac{\partial f}{\partial x_3} \\
\end{bmatrix}
=
\begin{bmatrix}
1 \\
-1 \\
-1 \\
\end{bmatrix}
$$

The gradient of $f$ gives the partial derivatives in each dimensions. The partial derivative $\frac{\partial f}{\partial x_1}$ gives us the slope of $f$ in the positive $x_1$ direction, $\frac{\partial f}{\partial x_2}$ gives us the slope of $f$ in the positive $x_2$ direction and $\frac{\partial f}{\partial x_3}$ gives us the slope in the positive $x_3$ direction.

Instead of fixing the direction, we can generalise the partial derivatives to compute the slope in any direction that we want. Like the partial derivative, we can define the directional derivative as follows:

\begin{align*}
D_{\mathbf{d}} f(\mathbf{x}) = \lim_{h \to 0} \frac{f(\mathbf{x} + h \mathbf{d}) - f(\mathbf{x})}{h}
\end{align*}

This means that we can dot the gradient with the direction vector $\mathbf{d}$ to find the directional derivative:

In [3]:
x1, x2, x3 = sy.symbols('x1, x2, x3')
f = Func(x1 - x2 - x3, (x1, x2, x3))
fg = f.gradient()
fg

⎡1 ⎤
⎢  ⎥
⎢-1⎥
⎢  ⎥
⎣-1⎦

The directional derivative:
$$
\frac{\partial f(\mathbf{x})}{\partial \mathbf{d}} 
= \nabla f(\mathbf{x})^T \mathbf{d}
=
[1, -1, -1]
\begin{bmatrix}
2 \\
3 \\
4 \\
\end{bmatrix}
= -5
$$

In [4]:
fg.T * sy.Matrix([[2], [3], [4]])

[-5]

---
<img src="figures/exam-jan-2018/ex2b.png" width="800" />















Since it is an optimisation problem with equality constraints, we can use the Lagrangian method.

The Lagrangian method is given as follows:
1. Define the Lagrangian function by incoporating the constraint into the objective function
$$
L(\mathbf{x}, \lambda) = f(\mathbf{x}) - \lambda ( h(\mathbf{x}) - c  )
$$
where $\lambda$ is the Lagrangian multiplier associated with the contraint. When the contraint holds i.e., when $h(\mathbf{x})=c$ then $L(\mathbf{x}, \lambda)=f(\mathbf{x})$
2. Since $L(\mathbf{x}, \lambda)$ is an unconstraint objective function, we can optimise it by finding its FONC i.e., taking the derivative and setting it zero.
3. Now, we get a system of linear equations. Solving it will yield the points where the objective function is maximised or minimised.

In [5]:
x1, x2, x3 = sy.symbols('x1, x2, x3')

f = x1 - x2 - x3
h = Fraction(1, 4) * x1**2 + Fraction(1, 9) * x2**2 + Fraction(1, 3) * x3**2 - 1
F = Func(f, (x1, x2, x3), constraints=[h])

In [6]:
points, lambdas = F.solve_lagrangian()

In [7]:
points

In [8]:
F.evalf(points)

Evaluting f(-1, 9/4, 3/4) = -4.00000000000000
Evaluting f(1, -9/4, -3/4) = 4.00000000000000


The function $f$ is minimum at point $(-1, 9/4, 3/4)$ and maximum at point $(1, -9/4, -3/4)$

---
# Exercise 3

Answer the following with ONE sentence per question.


**a) What is the main difference between the Newton and Quasi-Newton method?**

The Quasi-Newton method uses an approximation of the inverse of the Hessian instead of the true inverse which is done in the Newton method.



<img src="figures/lecture-10/quasi-newton-intro.png" width="600" />



----
**b) What is discrete optimization? Name one example of a well-known discrete optimization problem.**


When the input space (optimisation variables) is finite then we have a discrete optimisation, otherwise we have a continuous. The travelling salesman problem is an well-known example of a discrete optimisation problem.

---
**c) What is the main feature of global search methods?**


Global search methods have built-in mechanisms that allow them to escape local minima although they do not gurantee to find the global optimum.

---
<img src="figures/exam-jan-2018/ex3d.png" width="800" />















- Particle Swarm: Stochastic, Global Search, Population of candidate solution
- Simulated Annealing: Stochastic, Global Search
- Quasi-Newton: Deterministic
- Newton: Deterministic
- Genetic Algorithms: Stochastic, Global Search, Population-based
- Conjugate Gradient: Deterministic

---
<img src="figures/exam-jan-2018/ex3e.png" width="800" />


























---
<img src="figures/exam-jan-2018/ex3e1.png" width="800" />




Since the turle can move in three distinct directions, we can use 2 bits to represent each direction; 00 = Up, 01 = Right, 10 = Left. A turtle path  consists of 5 steps, so each path (candidate solution) can be represented as a chromosome of length 10. Here are some examples of chromosomes that corresponds to turtle paths:

- `00 01 01 00 10` corresponds to the first path  
- `00 10 00 00 10` corresponds to the second path
- `00 00 00 00 01` corresponds to the third path

---
<img src="figures/exam-jan-2018/ex3e2.png" width="800" />






Use the Manhattan distance as the objective function. Here are the costs of the three example paths:
- `cost(example1) = 2`
- `cost(example2) = 3`
- `cost(example3) = 0`

---
<img src="figures/exam-jan-2018/ex3e3.png" width="800" />






Suppose we have two parents; A and B. A's chromosome is `00 01 01 00 10` and B's chromosome is `01 00 00 01 01`

Single point crossover works as follows:
- Randomly pick a crossover position
- Generate two children by swapping the chromosomes of the two parents

For example, if we pick crossover position 4, then single-point crossover yields two children
- Child 1 with the chromosome `00 01 00 01 01`
- Child 2 with the chromosome `01 00 01 00 10`

---
<img src="figures/exam-jan-2018/ex3e4.png" width="800" />






One bit of Child 1's chromosome `00 01 00 01 01` may be flipped with a low probability to `00 01 00 01 00`.

---
<img src="figures/exam-jan-2018/ex3e5.png" width="800" />






In [9]:
def genetic_algorith(P, N, num_generations):
    for i in range(num_generations):
        compute_individual_fitness_of(P)
        mating_pool = select_mating_pool(P)
        offspring = cross_over(mating_pool)
        P = mutate(offspring)

1. Compute the fitness of each individual in the population. Fitness is based on the objective function value of that candidate solution.
2. Select individuals based on their fitness in the population. For example, we can select chromosomes into the mating pool with probabilities proportional to their fitness.
3. Perform crossover operation on individuals in the mating pool to create the next generation of individuals
4. Perform mutation i.e., alters bits in the chromosome with low probability
5. Use the offspring as the population in the next generation

---
# Exercise 4
<img src="figures/exam-jan-2018/ex4.png" width="800" />






In [10]:
X = np.array([[0, 1, 1, 3, 3, 4, 4],
              [1, 0, 1, 1, 2, 4, 2]])
l = np.array( [1, 1, 1, 2, 2, 2, 2])
X_test = np.array([[1, 3, 2],
                   [2, 0, 1]])
C1 = X[:,0:3]
C2 = X[:,3:]

In [11]:
from sklearn.metrics.pairwise import pairwise_distances

**a) The Nearest Class Centroid (NCC) classifier**

In [12]:
# Compute the class centroid for each class
mu1 = np.mean(C1, axis=1).reshape(-1, 1)
mu2 = np.mean(C2, axis=1).reshape(-1, 1)

In [13]:
# Compute distances between the centroids and the test samples
pair_dist_mu1 = pairwise_distances(X_test.T, mu1.T)
pair_dist_mu2 = pairwise_distances(X_test.T, mu2.T)
pair_dist = np.concatenate([pair_dist_mu1, pair_dist_mu2], axis=1)

In [14]:
# Classify using the lowest distance
np.argmin(pair_dist, axis=1)+1

array([1, 2, 1])

This means that $\mathbf{x}_8$ and $\mathbf{x}_{10}$ are assigned label 1 and $\mathbf{x}_9$ gets label 2.

---
**b) The Nearest Neighbor Classifier (using only one neighbor)**



In [15]:
X = np.array([[0, 1, 1, 3, 3, 4, 4],
              [1, 0, 1, 1, 2, 4, 2]])
X_test = np.array([[1, 3, 2],
                   [2, 0, 1]])

In [16]:
classifications = []
for i in range(X_test.shape[1]):
    x_i = X_test[:,i].reshape(-1, 1)
    # Compute the distance each sample in the training set
    pair_dist = pairwise_distances(X.T, x_i.T).reshape(-1)
    
    # Assign the label of the closest sample
    index = np.argmin(pair_dist, axis=0)
    label = l[index]
    
    classifications.append(label)
classifications

[1, 2, 1]

The samples $\mathbf{x}_8$ and $\mathbf{x}_{10}$ are assigned label 1 and $\mathbf{x}_9$ gets label 2.

---
<img src="figures/exam-jan-2018/ex4c.png" width="800" />






Compute the class-conditional probabilities in multiple steps.

Step 1: Compute the mean vector associated with each class:

In [17]:
m1 = np.mean(C1, axis=1).reshape(-1, 1)
m2 = np.mean(C2, axis=1).reshape(-1, 1)

Step 2: Compute $\lVert \mathbf{x}_i - \mathbf{m}_k \rVert^{-2}_2$

In [18]:
def calc_dist(samples):
    N = samples.shape[1]
    results = np.zeros(N)
    for i in range(N):
        x_i = samples[:,i]
        results[i] = 1/ np.dot(x_i, x_i)
    return results

In [19]:
C1_num = calc_dist(X_test - m1)
C2_num = calc_dist(X_test - m2)

Step 3: Compute the class-conditional probabilities $p(\mathbf{x} \mid c_k)$

In [20]:
p_x_c1 = C1_num / (C1_num + C2_num)
p_x_c2 = C2_num / (C1_num + C2_num)

Step 4: Compute the prior probabilities for the classes. We can compute the prior probability for each class $c_k$ as follows:

$$
P(c_k) = \frac{N_k}{\sum_l^K N_l}
$$
where $N_k$ is the number of samples in class $c_k$

In [21]:
N1 = C1.shape[1]
N2 = C2.shape[1]
N_sum = N1 + N2

P_c1 = N1/N_sum
P_c2 = N2/N_sum

Step 5: We can now compute conditional probability, which has the following formula (Bayes' formula):

$$
P(c_k \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid c_k) P(c_k)}{p(\mathbf{x})}
$$
where
$$
p(\mathbf{x}) = \sum_{k=1}^K p(\mathbf{x} \mid c_k) P(c_k)
$$

Since $p(\mathbf{x})$ is the same for when computing $P(c_k\mid \mathbf{x})$, it is not necessary to compute it when we want to use the conditional probability to classify new samples. Therefore, we omit the division:
$$
P(c_k \mid \mathbf{x}) = p(\mathbf{x} \mid c_k) P(c_k)
$$

In [22]:
P_c1_x = p_x_c1 * P_c1
P_c2_x = p_x_c2 * P_c2

# Create a matrix so we can easily compute argmin
P_ck_x = np.concatenate([[P_c1_x], 
                         [P_c2_x]])

Step 6: Once we have the condition probabilities, we can use Bayes' Decision Rule to classify our samples $X$:

In [23]:
labels = np.argmax(P_ck_x, axis=0)+1
out = 'Classification Result: \n$$\n'
for i, label in enumerate(labels):
    out += '\mathbf{{x}}_{{{}}} \\to {}  \\\\ \n'.format(i+8, label)
out += '$$'
HTML(out)

---
**d) Compare (qualitatively) the decision functions obtained by using the NCC classifier and the above
Bayes-based classification scheme.**

Their decision functions are almost the same. NCC classifier can be seen as a Bayes-based classifier with two assumptions:
1. The samples in each class follow a unimodal normal distribution with identity matrix as the covariant matrix, and
2. All classes have equal probability

Since the prior probability of class 2 is larger than class 1, the decision boundary for the Bayes-based classifier is moved slightly to the left compared to NCC.

---
# Exercise 5

<img src="figures/exam-jan-2018/ex5a.png" width="800" />






<img src="figures/exam-jan-2018/sol-ex5a.jpg" width="600" />






---
<img src="figures/exam-jan-2018/ex5b.png" width="800" />






To simplify the proof, we ignore the bias terms, use the identify function as the activation function for all neurons and have only a single neuron in the output layer.

Suppose we have a two-layer network with a single output. The output of this network is:
$$
o = \mathbf{w}^T \mathbf{x}
$$

Now, suppose we have a four-layer network (two hidden layers). The output of this network in steps.

The output neuron is computed:

$$
o_{ML} = \mathbf{w}^{(3)T} \mathbf{h}_2
$$

The $\mathbf{h}_2$ is computed as follows:

$$
\mathbf{h}_2 = \mathbf{W}^{(2)T} \mathbf{h}_1
$$

The $\mathbf{h}_1$ is computed as follows:

$$
\mathbf{h}_1 = \mathbf{W}^{(1)T} \mathbf{x}
$$

Combining the three expressions, we get:

$$
o_{ML} = \mathbf{w}^{(3)T} \mathbf{W}^{(2)T} \mathbf{W}^{(1)T} \mathbf{x}
$$

The expression above can be simplied:
$$
o_{L} = \mathbf{w}^T_{c} \mathbf{x}
$$
where $\mathbf{w}_{c} = \mathbf{w}^{(3)T} \mathbf{W}^{(2)T} \mathbf{W}^{(1)T}$

This shows that the four-layer network and two layer network are equivalent when the activation function for all neurons are linear.

---
<img src="figures/exam-jan-2018/ex5c.png" width="800" />






The result in b) shows that using only linear activation functions in an ANN means that the network is a linear function of the  input. This means it can only learn a linear decision function even though we have a network with large number of hidden layers. If our data is non-linear then we cannot model it. Therefore, non-linear activation functions are important in neural networks.

---
# Exercise 6

<img src="figures/exam-jan-2018/ex6a.png" width="800" />




<img src="figures/exam-jan-2018/sol-ex6a.png" width="400" />















---
<img src="figures/exam-jan-2018/ex6b.png" width="800" />




<img src="figures/exam-jan-2018/sol-ex6b.png" width="400" />















---
<img src="figures/exam-jan-2018/ex6c.png" width="800" />






The regions of ambiguity for one-vs-rest are coloured in grey:

<img src="figures/exam-jan-2018/sol-ex6c1.png" width="400" />















The region of ambiguity for one-vs-one is coloured in grey:

<img src="figures/exam-jan-2018/sol-ex6c2.png" width="400" />













